In [34]:
import tensorflow as tf
import tensorflow_hub as hub
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
from tensorflow.keras.preprocessing import image

In [35]:
data_dir_training = "/Users/aaryangulia/Downloads/archive/asl_alphabet_train/asl_alphabet_train"

In [36]:
# Define the number of classes
num_classes = len(os.listdir(data_dir_training)) - 1

# Define the image size and batch size
img_size = (224, 224)
batch_size = 32

# Use ImageDataGenerator for data loading and augmentation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    validation_split=0.2,  # Split data into training and validation sets
    rotation_range=20,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in radians)
    zoom_range=0.2,  # Randomly zoom images by up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

In [37]:
train_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # For multi-class classification
    subset='training'  # Specify this as training data
)

# Create validation data generator
validation_generator = datagen.flow_from_directory(
    data_dir_training,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'  # Specify this as validation data
)

Found 69600 images belonging to 29 classes.
Found 17400 images belonging to 29 classes.


In [38]:
# Load the pre-trained MobileNetV2 model without the top classification layer
base_model = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
base_model.trainable = False

# Add custom classification layers for your specific task
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # num_classes is the number of classes in your dataset
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Now, you can train the model with your own dataset using model.fit(...)

In [39]:
model.load_weights('model_weights.h5')

In [40]:
# Print model summary to verify
print("\n Model summary: ")
print(model.summary())



 Model summary: 
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Func  (None, 7, 7, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d_2  (None, 1280)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 29)                37149     
                                                                 
Total params: 2295133 (8.76 MB)
Trainable params: 37149 (145.11 KB)
Non-trainable params: 2257984 (8.61 MB)
_________________________________________________________________
None


In [41]:
# Define a function to load and preprocess an image
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)  # Load the image
    img_array = image.img_to_array(img)  # Convert image to numpy array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Rescale pixel values to [0, 1] as done during training
    return img_array

# Load and preprocess the image
image_path = '/Users/aaryangulia/Downloads/archive-2/Train_Alphabet/D/0ad4fbba-93fc-4bee-8056-0af4f58837d3.rgb_0000.png'
processed_image = load_and_preprocess_image(image_path)

In [42]:
# Make a prediction using the pre-trained model
predictions = model.predict(processed_image)

# Get the predicted class label
predicted_class_idx = np.argmax(predictions[0])

# Decode the class index to get the class label (assuming you have a list of class labels)
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_idx]

# Get the confidence score of the prediction
confidence = predictions[0][predicted_class_idx]

# Print the results
print(f'Predicted class: {predicted_class_label}')
print(f'Confidence: {confidence:.2f}')

1/1 [==============================] - 2s 2s/step
Predicted class: I
Confidence: 0.97


In [43]:
# Save the Keras model
tf.keras.models.save_model(model, 'epoch3_model')

INFO:tensorflow:Assets written to: epoch3_model/assets


INFO:tensorflow:Assets written to: epoch3_model/assets


In [18]:
print(class_labels)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']
